# Chapter 3 -- Bifurcation

In [47]:
import warnings
import numpy as np
import pandas as pd
import itertools as it
import plotly.express as px
import plotly.graph_objects as go
%matplotlib inline

In [4]:
def derivative(func, x0, h, **kwargs):
    return (func(x=x0+h, **kwargs) - func(x=x0-h, **kwargs))/(2 * h)

In [5]:
def rx2df(func, func_ans, r, x, **kwargs):
    r_x_seq = np.array(np.meshgrid(r, x)).T.reshape(-1, 2)
    x_dot = func(r = r_x_seq[..., 0], x = r_x_seq[..., 1], **kwargs)
    x_derivative = derivative(func, x0=r_x_seq[..., 1], h=1e-6, r=r_x_seq[..., 0], **kwargs)
    x_answers = func_ans(r_x_seq[..., 0], **kwargs)

    df = pd.DataFrame()
    df["r"] = r_x_seq[..., 0]
    df["x"] = r_x_seq[..., 1]
    df["x_dot"] = x_dot
    df["x_diff"] = x_derivative
    for i in range(x_answers.shape[-1]):
        df[f"ans{i+1}"] = x_answers[..., i]
    
    return df

In [7]:
def df2vector_field(df, title, filename):
    fig = px.line(df, 
        x="x", 
        y="x_dot", 
        animation_frame="r", 
        width=1000, 
        height=1000, 
        title=title)
    fig.write_html(filename)

In [56]:
def df2bifurcation_diagram(func, df, filename, num_ans=2, **kwargs):

    colors = np.array([derivative(func, df[f"ans{i}"].values, h=1e-6, r=df["r"], **kwargs) for i in range(1, num_ans+1)])
    colors = np.where(np.isnan(colors), 0, colors)

    fig = go.Figure()

    
    for i in range(1, num_ans+1):
        fig.add_trace(go.Scatter(
            x=df["r"],
            y=df[f"ans{i}"],
            marker=dict(
                size=2,
                cmax=colors.max(),
                cmin=colors.min(),
                color=colors[i-1],
                colorbar=dict(
                    title="Unstability"
                ),
                colorscale="Bluered"
            ),
            mode="markers",
            showlegend=False))

    fig.update_layout(
        width=1000, 
        height=600)
    fig.update_xaxes(title_text="r")
    fig.update_yaxes(title_text="Fixed Points")
    fig.write_image(filename)

## 3.1 Saddle-Node Bifurcation

**ChatGPT's description**

In the context of a one-dimensional dynamical system described by a scalar equation, the saddle-node bifurcation can be observed when a parameter of the system is varied. Initially, the system has two equilibrium points—one stable and one unstable. As the parameter crosses a critical value, these two equilibrium points collide and annihilate each other, giving rise to a single new equilibrium point, which is either stable or unstable depending on the specific dynamics of the system. 

### 3.1.1 Normal Form of Saddle-Node Bifurcation

$$\dot{x} = r \pm x^2$$

In [39]:
def normal_saddle_node(r, x, is_plus = True):
    if is_plus:
        return r + x*x
    else:
        return r - x*x

In [40]:
def normal_saddle_node_ans(r, is_plus = True):
    res = np.zeros((r.shape[0], 2))
    if is_plus:
        for i in range(len(res)):
            answer = np.roots([1, 0, r[i]])
            if type(answer[0]) == np.complex128:
                res[i] = np.inf
            else:
                res[i] = answer
    else:
        for i in range(len(res)):
            answer = np.roots([-1, 0, r[i]])
            if type(answer[0]) == np.complex128:
                res[i] = np.inf
            else:
                res[i] = answer
    return res

### 3.1.2 Data Peparation
Data list will be created in an area with $ -5 \leq r \leq 5, -5 \leq x \leq 5$.

In [41]:
r = np.linspace(-5, 5, 300)
x = np.linspace(-5, 5, 300)
is_plus = True

df_saddle_node = rx2df(normal_saddle_node, normal_saddle_node_ans, r, x, is_plus=True)

df_saddle_node

,r,x,x_dot,x_diff,ans1,ans2
0,-5.0,-5.000000,20.000000,5.000000,2.236068,-2.236068
1,-5.0,-4.966555,19.666670,4.933110,2.236068,-2.236068
2,-5.0,-4.933110,19.335578,4.866221,2.236068,-2.236068
3,-5.0,-4.899666,19.006723,4.799331,2.236068,-2.236068
4,-5.0,-4.866221,18.680104,4.732441,2.236068,-2.236068
...,...,...,...,...,...,...
89995,5.0,4.866221,28.680104,-4.732441,inf,inf
89996,5.0,4.899666,29.006723,-4.799331,inf,inf
89997,5.0,4.933110,29.335578,-4.866221,inf,inf
89998,5.0,4.966555,29.666670,-4.933110,inf,inf


In [54]:
df2vector_field(df_saddle_node, title='r + x*x', filename="../../../figures/chap3-saddle-node-vector-field.html")

In [42]:
df2bifurcation_diagram(normal_saddle_node, df_saddle_node, "../../../figures/chap3-saddle-node-bifurcation-diagram.pdf", is_plus=is_plus)

## 3.2 Transcritical Bifuraction

**ChatGPT's description**

In a one-dimensional dynamical system described by a scalar equation, the transcritical bifurcation can be observed when a parameter crosses a critical value. Prior to the bifurcation, there are two equilibrium points—one stable and one unstable. As the parameter changes, the stable equilibrium point and the unstable equilibrium point move towards each other. At the critical value, they exchange their stability properties. The previously stable equilibrium point becomes unstable, and the previously unstable equilibrium point becomes stable.

### 3.2.1 Normal Form of Transcritical Bifurcation

$$\dot{x} = rx - x^2$$

In [45]:
def normal_transcritical(r, x):
    return r*x - x*x

In [46]:
def normal_transcritical_ans(r):
    res = np.zeros((r.shape[0], 2))
    for i in range(len(res)):
        answer = np.roots([-1, r[i], 0])
        res[i] = answer
    return res

### 3.2.2 Data Preparation

Data list will be created in an area with $ -5 \leq r \leq 5, -10 \leq x \leq 10$.

In [50]:
r = np.linspace(-5, 5, 300)
x = np.linspace(-10, 10, 300)

df_transcritical = rx2df(normal_transcritical, normal_transcritical_ans, r, x)

df_transcritical

,r,x,x_dot,x_diff,ans1,ans2
0,-5.0,-10.000000,-50.000000,15.000000,-5.0,0.0
1,-5.0,-9.933110,-49.001130,14.866221,-5.0,0.0
2,-5.0,-9.866221,-48.011208,14.732441,-5.0,0.0
3,-5.0,-9.799331,-47.030235,14.598662,-5.0,0.0
4,-5.0,-9.732441,-46.058210,14.464883,-5.0,0.0
...,...,...,...,...,...,...
89995,5.0,9.732441,-46.058210,-14.464883,5.0,0.0
89996,5.0,9.799331,-47.030235,-14.598662,5.0,0.0
89997,5.0,9.866221,-48.011208,-14.732441,5.0,0.0
89998,5.0,9.933110,-49.001130,-14.866221,5.0,0.0


In [56]:
df2vector_field(df_transcritical, title="r*x - x*x", filename="../../../figures/chap3-transcritical-vector-field.html")

In [52]:
df2bifurcation_diagram(normal_transcritical, df_transcritical, "../../../figures/chap3-transcritical-bifurcation-diagram.pdf")

## 3.3 Pitchfork Bifurcation

### 3.3.1 Normal Form of Pitchfork Bifurcation

$$\dot{x} = rx - x^3$$

In [1]:
def normal_pitchfork(r, x):
    return r * x - x**3

In [50]:
def normal_pitchfork_ans(r):
    warnings.simplefilter('ignore')
    res = np.zeros((r.shape[0], 3))
    for i in range(len(res)):
        answer = np.roots([-1, 0,  r[i], 0])
        is_complex = np.iscomplex(answer)
        answer[is_complex] = np.inf
        res[i] = answer
    warnings.resetwarnings()
    return res

### 3.3.2 Data Preparation

Data list will be created in an area with $ -5 \leq r \leq 5, -10 \leq x \leq 10$.

In [51]:
r = np.linspace(-5, 5, 300)
x = np.linspace(-3, 3, 300)

df_pitchfork = rx2df(normal_pitchfork, normal_pitchfork_ans, r, x)

df_pitchfork

,r,x,x_dot,x_diff,ans1,ans2,ans3
0,-5.0,-3.000000,42.000000,-32.000000,inf,inf,0.0
1,-5.0,-2.979933,41.361476,-31.640004,inf,inf,0.0
2,-5.0,-2.959866,40.730151,-31.282424,inf,inf,0.0
3,-5.0,-2.939799,40.105978,-30.927260,inf,inf,0.0
4,-5.0,-2.919732,39.488907,-30.574513,inf,inf,0.0
...,...,...,...,...,...,...,...
89995,5.0,2.919732,-10.291582,-20.574513,2.236068,-2.236068,0.0
89996,5.0,2.939799,-10.707984,-20.927260,2.236068,-2.236068,0.0
89997,5.0,2.959866,-11.131489,-21.282424,2.236068,-2.236068,0.0
89998,5.0,2.979933,-11.562144,-21.640004,2.236068,-2.236068,0.0


In [52]:
df2vector_field(df_pitchfork, title="r*x - x**3", filename="../../../figures/chap3-pitchfork-vector-field.html")

In [55]:
df2bifurcation_diagram(normal_pitchfork, df_pitchfork, "../../../figures/chap3-pitchfork-bifurcation-diagram.pdf", num_ans=3)